# Bucket brigade
---


In [ ]:
import random
import numpy as np
import simpy
from pylab import plot, show, bar
import matplotlib.pyplot as plt
plt.style.use("ggplot")

In [ ]:
class factory(object):
    """
    """
    def __init__(self, env,items_number,workers_number,collect_rate,forward_rate,back_rate,batches):
        self.env = env
        self.item = []
        self.workers = []
        self.batch_order = batches
        self.finish_all = env.event()
        self.time_to_finish = 0
        for i in range(items_number):
            self.item.append(simpy.Resource(env, 1))
        for j in range(workers_number):
            self.workers.append(worker(j, collect_rate[j],forward_rate[j],back_rate[j]))

In [ ]:
class box():
    def __init__(self, capacity,batch,worker):
        self.capacity = capacity            #capacity of the box
        self.batch = batch                  #dictinary of items and their quantity
        self.finished = env.event()         #batch is finished?
        self.status = worker                #which worker is with the batch box
        
    def create_forward(self,env,factory):
        i = self.status.name()
        t = max(0,np.random.normal(self.status.forward_rate[0],self.status.forward_rate[1]))
        if self.status.location<self.batch[-1]:
            with self.item[self.status.location+1].request() as request:
                Result = yield request| factory.workers[i+1].location==self.status.location 
                if request in results:
                    if self.status.location in batch:
                        yield env.timeout(t)
                        self.status.location+=1
                        yield env.timeout(max(0,np.random.normal(self.status.collect_rate[0],self.status.collect_rate[1])))
                        if factory.workers[i+1].location!=self.status.location:
                            env.process(create_forward(self,env,factory))
                        else:
                            env.process(create_back(self.status,env,factory))
                            self.status = factory.workers[i+1]
                            env.process(create_forward(self,env,factory))
                    else:
                        yield env.timeout(t)
                        self.status.location+=1
                        env.process(create_forward(self,env,factory))
                else:
                    env.process(create_back(self.status))
                    self.status = factory.workers[i+1]
                    env.process(create_forward(self,env,factory))
        else:
            self.finished.succeed()
            if len(factory.batch_order)==0:
                factory.finish_all.succeed()
            else:
                env.process(create_back(self.status,env,factory))        

In [ ]:
class worker():
    def __init__(self, name, collect_rate,forward_rate,back_rate):
        self.name = name
        self.collect_rate = collect_rate
        self.forward_rate = forward_rate
        self.back_rate = back_rate             
        self.location = 0                      #worker location by axis x
        self.finish = env.event()
        self.with_box = None
        
    def create_back(self,env,factory):
        t = max(0,np.random.normal(self.back_rate[0],self.back_rate[1]))
        i = self.name()
        if self.location!=0:
            with factory.item[self.location-1].request() as request:
                Result = yield request| factory.workers[i-1].location==self.location
                if request in results:
                    yield env.timeout(t)
                    self.location -=1
                else:
                    env.process(create_back(self,env,factory))
                    env.process(create_forward(self.with_box,env,factory))
        else:
            if len(factory.batch_order)>0:
                k = factory.batch_order.pop()
                bucket = box(30,k,self)
                self.with_box = bucket
                env.process(create_forward(bucket,env,factory))
            else:
                self.finish.succeed()               

In [ ]:
def setup(env,factory,workers_number):
    """
    """
    for w in range(workers_number):
        new_worker = worker(w+workers_number, collect_rate[w],forward_rate[w],back_rate[w])
        k = Factory.batch_order.pop()
        bucket = box(30,k,new_worker)
        new_worker.with_box = bucket
        env.process(create_forward(bucket,env,factory))

In [ ]:
def main_sim():    
    # Create an environment and start the setup process
    RANDOM_SEED = 42
    # ----------Setup and start the simulation-------------------
    items_number = 5  # Number of items
    workers_number = 2  # Number of workers
    collect_rate = []
    forward_rate = []
    back_rate = []
    batches = []
    
    # Create the factory
    Factory = factory(env,items_number,workers_number,collect_rate,forward_rate,back_rate,batches)

    # start the simulation
    print('Factory')
    random.seed(RANDOM_SEED)  # This helps reproducing the results

    env = simpy.Environment()
    env.process(setup(env,Factory,workers_number))
    # Execute!
    env.run(until=Factory.finish_all)
    return Factory.time_to_finish